In [1]:
# Importamos módulos necesarios
import os  # Para manejar variables de entorno
from dotenv import load_dotenv  # Para cargar las variables de entorno desde un archivo .env
from openai import OpenAI  # Cliente para usar la API de OpenAI
import requests  # Para hacer solicitudes HTTP (en este caso a la API de Star Wars)
import json  # Para manipular datos en formato JSON

# Carga las variables de entorno (como la clave API de OpenAI)
load_dotenv()

# Inicializamos el cliente de OpenAI con la clave API obtenida desde el archivo .env
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Función que busca información sobre un personaje de Star Wars usando la SWAPI
def get_star_wars_character(name: str) -> str:
    print(f"Buscando personaje: {name}")  # Imprime el nombre del personaje a buscar
    url = f"https://swapi.py4e.com/api/people/?search={name}"  # URL de búsqueda en la API de Star Wars
    response = requests.get(url)  # Hacemos la solicitud a la API
    data = response.json()  # Convertimos la respuesta en un diccionario Python

    if data["count"] == 0:  # Si no se encuentra ningún personaje con ese nombre
        return f"No se encontró información sobre el personaje '{name}'."
    # Si se encuentra, devolvemos el primer resultado, bien formateado
    return json.dumps(data["results"][0], indent=2)

# Simulación de una conversación con el asistente
messages = [
    {
        "role": "system",  # Mensaje que define el comportamiento del asistente
        "content": "Eres un asistente experto en el universo de Star Wars. Usa la función get_star_wars_character para buscar información sobre personajes."
    },
    {
        "role": "user",  # Mensaje del usuario preguntando por un personaje
        "content": "¿Quién es Luke Skywalker?"
    }
]

# Definición de las funciones que el modelo puede usar
functions = [
    {
        "type": "function",  # Especificamos que se trata de una herramienta tipo función
        "function": {
            "name": "get_star_wars_character",  # Nombre de la función
            "description": "Obtiene información sobre un personaje de Star Wars",  # Descripción de la función
            "parameters": {  # Parámetros requeridos por la función
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Nombre del personaje de Star Wars"
                    }
                },
                "required": ["name"]  # El nombre es obligatorio
            },
            "output": {
                "type": "string",
                "description": "Información sobre el personaje de Star Wars"
            }
        }
    }
]

# Primer llamado al modelo con la conversación y la función disponible
response = client.chat.completions.create(
    model="gpt-4o",  # Modelo usado: GPT-4o
    messages=messages,  # Mensajes de la conversación
    tools=functions  # Funciones que puede utilizar el asistente
)

# Guardamos la respuesta del asistente
assistant_message = response.choices[0].message

print("Respuesta del asistente")
print(assistant_message)

# Si el asistente decidió usar una función:
if assistant_message.tool_calls:
    for tool_call in assistant_message.tool_calls:
        if tool_call.type == "function":
            function_name = tool_call.function.name
            # Convertimos los argumentos recibidos en JSON a un diccionario
            function_args = json.loads(tool_call.function.arguments)

            # Si la función es get_star_wars_character
            if function_name == "get_star_wars_character":
                print(f"El asistente está llamando a la función get_star_wars_character")
                # Llamamos la función local con el nombre del personaje
                character_info = get_star_wars_character(
                    name=function_args.get("name")
                )

                # Añadimos a la conversación la llamada a la función y su resultado
                messages.append(assistant_message)
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": character_info
                })

# Segundo llamado al modelo, ahora con la información del personaje ya obtenida
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)

# Obtenemos la respuesta final del asistente basada en la información de la función
final_reply = second_response.choices[0].message.content

print("Respuesta final del asistente:")
print(final_reply)


Respuesta del asistente
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_X6YS850gyMInWEfhSs4l7S2b', function=Function(arguments='{"name":"Luke Skywalker"}', name='get_star_wars_character'), type='function')], annotations=[])
El asistente está llamando a la función get_star_wars_character
Buscando personaje: Luke Skywalker
Respuesta final del asistente:
Luke Skywalker es uno de los personajes principales del universo de Star Wars. Aquí tienes algunos detalles sobre él:

- **Altura:** 172 cm
- **Peso:** 77 kg
- **Color de cabello:** Rubio
- **Color de piel:** Clara
- **Color de ojos:** Azules
- **Año de nacimiento:** 19 ABY (Antes de la Batalla de Yavin)
- **Género:** Masculino
- **Homeworld:** [Tatooine](https://swapi.py4e.com/api/planets/1/)
- **Especies:** Humano

**Aparece en las siguientes películas:**
- [Episodio IV: Una Nueva Esperanza](https://swapi.py4e.com/api/films/1/)
- [Epis